<a href="https://colab.research.google.com/github/geoffcorvera/deeplearning-spring2021/blob/main/MNIST_Fashion_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np 
import matplotlib.pyplot as plt 
from tensorflow.keras import layers
from keras.datasets.fashion_mnist import load_data

# Define Generator Model


*   Input: 100-dimensional sample from Gaussian latent space

*   First dense layer: 100 neurons (followed by BatchNormalization & LeakyRELU)
*   Intermediate layer: 500 dense (followed by BatchNormalization & LeakyRELU)
*   Last dense layer: 784 (neurons) (sigmoid activation)
*   Reshape into desired image shape 

*   Output: Generated image (28x28)



In [2]:
img_shape = (28,28,1)

# TODO: Finesse the number of neurons for up/sampling
# Input should have enough space for low-res version of generated image
n_initial = 100
n_intermediate = 500
n_final = 784

def makeDenseGenerator(latent_dims=100):
  latent_sample = keras.Input(latent_dims)
  # Input a sample from n-dimensional latent space (typically from Gaussian)
  x = layers.Dense(n_initial)(latent_sample)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU()(x)
  # Upsample
  x = layers.Dense(n_intermediate)(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU()(x)
  # Use sigmoid for bounded pixel intensities
  pixel_intensities = layers.Dense(n_final, activation='sigmoid')(x)
  # x = layers.BatchNormalization()(x)
  # x = layers.LeakyReLU()(x)
  # Reshape into image dimensions
  generated_img = layers.Reshape(img_shape)(pixel_intensities)

  generator = keras.Model(latent_sample, generated_img, name='generator')
  generator.summary()

  return generator


# TODO: Use transpose convolution (+ bi-linear interp) for up-sampling?
def createConvGenerator(latent_dims=100):
  print('not implemented')

In [3]:
g_model = makeDenseGenerator()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               50500     
_________________________________________________________________
batch_normalization_1 (Batch (None, 500)               2000      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 500)               0 

# Define Descriminator Model

Tries to mirror the generator network.

In [4]:
def makeDiscriminator(input_shape=(28, 28, 1)):
  input_img = keras.Input(input_shape)

  x = layers.Flatten()(input_img)
  x = layers.Dense(n_final)(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU()(x)

  x = layers.Dense(n_intermediate)(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU()(x)

  x = layers.Dense(n_initial)(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU()(x)
  
  p_real = layers.Dense(1, activation='sigmoid')(x)

  discriminator = keras.Model(input_img, p_real, name='discriminator')

  adam = keras.optimizers.Adam(learning_rate=.0002, beta_1=0.5)
  discriminator.compile(
    optimizer=adam,
    loss='binary_crossentropy',
    metrics=['accuracy']
  )

  discriminator.summary()

  return discriminator  

In [5]:
d_model = makeDiscriminator()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 784)               615440    
_________________________________________________________________
batch_normalization_2 (Batch (None, 784)               3136      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 500)               392500    
_________________________________________________________________
batch_normalization_3 (Batch (None, 500)             

# Define GAN

In [6]:
def makeGAN(g_model, d_model, latent_dims=100):
  d_model.trainable = False

  latent_sample = keras.Input(shape=(latent_dims,1))
  generated_img = g_model(latent_sample)
  p_real = d_model(generated_img)
  gan = keras.Model(latent_sample, p_real, name='gan')

  adam = keras.optimizers.Adam(learning_rate=.0002, beta_1=0.5)
  gan.compile(optimizer=adam, loss='binary_crossentropy')
  gan.summary()

  return gan

In [ ]:
gan = makeGAN(g_model, d_model)

# Load & Preprocess MNIST Fashion Dataset

We load the training and test data as numpy arrays for the MNIST fashion dataset. The pixel intensities of the training and test images are rescaled to [0,1] by dividing intensities by max value (255). An extra dimension is added to each example, to specify image is single-channel.

In [7]:
def load_mnist():
  (trainx, _), (testx, _) = load_data()

  trainx = np.expand_dims(trainx, axis=-1)
  testx = np.expand_dims(testx, axis=-1)

  trainx = trainx / 255.
  testx = testx / 255.

  print('Train summary', trainx.shape, trainx.dtype)
  print('Test summary', testx.shape, testx.dtype)

  return trainx, testx

In [8]:
train_set, test_set = load_mnist()

Train summary (60000, 28, 28, 1) float64
Test summary (10000, 28, 28, 1) float64


# Generating Data for Training
Our goal is to train the discriminator network to output the probability that an input image is real. To this end, we need to train the discriminator on a combination of real and fake/generated images. We give the real examples a target of 1, and fake/generated examples a target of 0.

In [9]:
def generate_real_samples(dataset, n_samples):
  # randomly sample examples from the dataset
  indices = np.random.randint(0, len(dataset), n_samples),
  samples = dataset[indices]
  labels = np.ones((n_samples,1))

  assert samples.shape[0] == labels.shape[0]
  assert samples.shape[0] == n_samples
  
  return samples, labels

# Make n_samples number of fake images (28x28)
def generate_fake_samples(n_samples, g_model=None):
  if (g_model):
    z = generate_latent_points(n_samples)
    X = g_model(z)
  else:
    X = np.random.rand(28*28*n_samples)
    X = X.reshape((n_samples,28,28,1))

  y = np.zeros((n_samples,1))
  assert X.shape[0] == y.shape[0]

  return X, y

def generate_latent_points(n_samples, latent_dims=100):
  X = np.random.rand(n_samples * latent_dims)
  X = X.reshape(n_samples, latent_dims)
  return X

# Training Network

In [13]:
def train_discriminator(d_model, g_model, dataset, batch_size=32, n_iter=10):
  for i in range(n_iter):
    real_x, real_y = generate_real_samples(dataset, int(batch_size/2))
    fake_x, fake_y = generate_fake_samples(int(batch_size/2), g_model)
    real_acc = d_model.train_on_batch(real_x, real_y)
    fake_acc = d_model.train_on_batch(fake_x, fake_y)

# TODO: set epochs to 100
def train(gan, g_model, d_model, dataset, batch_size=32, n_epochs=2):
  n_batches = len(dataset) // batch_size
  n_half = batch_size // 2

  for i in range(n_epochs):
    for b in range(n_batches):
      x_fake, y_fake = generate_fake_samples(n_half, g_model)
      x_real, y_real = generate_real_samples(dataset, n_half)
      # Combine real and fake into single training batch
      X = np.vstack([x_fake, x_real])
      y = np.vstack([y_fake, y_real])

      # shuffle batch
      mask = np.arange(X.shape[0])
      np.random.shuffle(mask)
      X = X[mask]
      y = y[mask]

      loss_d = d_model.train_on_batch(X,y)
      
      x_gan = generate_latent_points(batch_size)
      y_gan = np.ones((batch_size,1))
      loss_gan = gan.train_on_batch(x_gan, y_gan)

In [14]:
train(gan, g_model, d_model, train_set)

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 784)               615440    
_________________________________________________________________
batch_normalization_15 (Batc (None, 784)               3136      
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 784)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 500)               392500    
_________________________________________________________________
batch_normalization_16 (Batc (None, 500)             

In [ ]:
# TODO: Generate some images w/ trained generator